In [ ]:
!pip install datasets
!pip install img2dataset
!wget https://huggingface.co/datasets/huggingface-projects/sd-multiplayer-data/resolve/main/rooms_data.db

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 32.9 MB/s 
     |████████████████████████████████| 182 kB 16.1 MB/s 
     |████████████████████████████████| 212 kB 38.1 MB/s 
     |████████████████████████████████| 127 kB 43.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 133 kB 10.1 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 26.7 MB 2.0 MB/s 
     |████████████████████████████████| 1.8 MB 39.9 MB/s 
     |████████████████████████████████| 173 kB 40.6 MB/s 
     |████

In [ ]:
%cd /content/
import sqlite3
import re
import itertools

f = open("path.txt", "w")
ff = open("prompt.txt", "w")
fff = open("time.txt","w")
# Create a SQL connection to our SQLite database
con = sqlite3.connect("/content/rooms_data.db")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT key FROM rooms_data;'):
  s = "https://d26smi9133w0oo.cloudfront.net/" + str(row[0]) + "\n"
  f.write(s)

for row in cur.execute('SELECT time FROM rooms_data;'):  
  s = str(row[0])
  s = " |" + s[5:16] + "|\n"
  fff.write(s)

for row in cur.execute('SELECT prompt FROM rooms_data;'):  
  s = str(row[0])
  s = s.replace("jpg", "")
  s = s.replace("webp", "")
  words = s.split()
  s = (" ".join(words[1:])) + "\n" 
  ff.write(s)

f = open("path.txt", "r")
line = open("path.txt", "r")
ff = open("prompt.txt", "r")
fff = open("time.txt","r")
lines = line.readlines()

rfintext = open("realfinal.txt", "w")
rfintext.write("url,caption\n")

for Line in lines:
  path = f.readline()
  path = (path.replace("\n", ""))
  prompt = ff.readline()
  prompt = (prompt.replace("\n", ""))
  time = fff.readline()
  time = (time.replace("\n", ""))
  fin = path + ',"' + str(prompt) + str(time) + '"\n'
  rfintext.write(fin)

# Be sure to close the connection
con.close()

/content


In [ ]:
# Review
%cd /content/
!head -n 1 prompt.txt
!head -n 3 realfinal.txt
!head -n 1 path.txt
!wc path.txt -l

/content
art novo budapest
url,caption
https://d26smi9133w0oo.cloudfront.net/room-0/1666985352-k6ZxRj6A4atUmJQzq3rjsz-2080_1056-art-novo-budapest.jpg,"art novo budapest |10-28T19:29|"
https://d26smi9133w0oo.cloudfront.net/room-0/1666985427-M6NKEivSK9qxnXA2XDPqh4-1760_1056-alion-attacking-city-from-space.jpg,"alion attacking city from space |10-28T19:30|"
https://d26smi9133w0oo.cloudfront.net/room-0/1666985352-k6ZxRj6A4atUmJQzq3rjsz-2080_1056-art-novo-budapest.jpg
200953 path.txt


In [ ]:
%cd /content/
!mkdir /content/imagefolder
%cd /content/imagefolder

/content
mkdir: cannot create directory ‘/content/imagefolder’: File exists
/content/imagefolder


In [ ]:
!img2dataset --url_list "/content/realfinal.txt" --input_format "csv" --url_col "url" --caption_col "caption" --thread_count=64 --image_size=256

Starting the downloading of this file
Sharding file number 1 of 1 called /content/realfinal.txt
0it [00:00, ?it/s]File sharded in 21 shards
1it [02:08, 128.44s/it]worker  - success: 0.993 - failed to download: 0.007 - failed to resize: 0.000 - images per sec: 82 - count: 10000
total   - success: 0.993 - failed to download: 0.007 - failed to resize: 0.000 - images per sec: 82 - count: 10000
2it [04:12, 125.93s/it]worker  - success: 0.997 - failed to download: 0.003 - failed to resize: 0.000 - images per sec: 81 - count: 10000
total   - success: 0.995 - failed to download: 0.005 - failed to resize: 0.000 - images per sec: 81 - count: 20000
3it [06:19, 126.27s/it]worker  - success: 0.994 - failed to download: 0.006 - failed to resize: 0.000 - images per sec: 79 - count: 10000
total   - success: 0.995 - failed to download: 0.005 - failed to resize: 0.000 - images per sec: 80 - count: 30000
4it [08:29, 127.72s/it]worker  - success: 0.996 - failed to download: 0.004 - failed to resize: 0.000

In [ ]:
import glob
from datasets import *
from datasets import load_dataset
textdataset = sorted(glob.glob("/content/imagefolder/images/*****/*.txt"))
text = []
for i in textdataset:
  textfile = open(i, "r")
  textcontent = textfile.readline()
  textcontent = textcontent.replace('"', "")
  textcontent = textcontent.replace('\n', "")
  text.append(textcontent)
ims = sorted(glob.glob('/content/imagefolder/images/*****/*.jpg'))
d = {'image':ims, "text":text}
dataset = Dataset.from_dict(d).cast_column("image", Image()).cast_column("text",Value(dtype='string'))
print(dataset)

Casting the dataset:   0%|          | 0/199 [00:00<?, ?ba/s]

Dataset({
    features: ['image', 'text'],
    num_rows: 198771
})


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
dataset.push_to_hub('-INSERTUSER0-/OSD-Dataset')

  0%|          | 0/14 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

...

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]